In [2]:

import sys
import os

project_root = os.path.abspath("..")
sys.path.append(project_root)

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from config import *
from Data.raw_data_loader import load_raw_crypto_csv
from Models.pair_trading import build_spread, pair_signal_zscore
from Models.ou import ou_signal_on_spread
from Backtest.engine import run_backtest
from Backtest.metrics import build_summary_table


In [3]:
df = load_raw_crypto_csv(DATA_PATH)

df = df.loc["2017-11-12":].copy()
btc = df["BTC-USD_close"].astype(float)
eth = df["ETH-USD_close"].astype(float)

mask = (btc > 0) & (eth > 0)
mask &= (btc.pct_change().abs() < 1.0)
mask &= (eth.pct_change().abs() < 1.0)
df = df.loc[mask].copy()

spread = build_spread(df, window=PAIR_BETA_WINDOW)

print("Data length:", len(df))
print("Spread length:", len(spread))


Data length: 3010


In [5]:
from statsmodels.tsa.stattools import adfuller

adf_stat, pvalue, *_ = adfuller(spread.dropna())

print("ADF stat:", adf_stat)
print("p-value:", pvalue)

ADF stat: -0.9523989641796868
p-value: 0.7702276016236705


In [ ]:
z_pos = pair_signal_zscore(
    spread,
    window=60,
    entry_z=2.0,
    exit_z=0.5,
)

z_results = run_backtest(
    price_series=spread,
    position=z_pos,
    fee_bps=FEE_BPS,
    slippage_bps=SLIPPAGE_BPS,
    leverage_cap=LEVERAGE_CAP,
)


In [ ]:

ou_pos = ou_signal_on_spread(
    spread,
    window=90,
    entry_z=1.5,
    exit_z=0.3,
    long_short=True,
)

# Trend on spread
trend_df = pd.DataFrame({"spread": spread})
trend_pos = trend_signal(
    trend_df,
    price_column="spread",
    fast_window=20,
    slow_window=90,
    long_only=False,
    leverage_aggressive=1.0,
    leverage_neutral=0.0,
    leverage_defensive=-1.0,
)

ou_results = run_backtest(
    price_series=spread,
    position=ou_pos,
    fee_bps=FEE_BPS,
    slippage_bps=SLIPPAGE_BPS,
    leverage_cap=LEVERAGE_CAP,
)


In [ ]:

def pair_portfolio_returns(df: pd.DataFrame, beta_window: int, position: pd.Series) -> pd.Series:
    log_b = np.log(df["BTC-USD_close"].astype(float))
    log_e = np.log(df["ETH-USD_close"].astype(float))

    minp = min(beta_window, max(30, beta_window // 3))
    cov = log_b.rolling(beta_window, min_periods=minp).cov(log_e)
    var = log_e.rolling(beta_window, min_periods=minp).var()
    beta = (cov / var).replace([np.inf, -np.inf], np.nan).ffill().fillna(0.0)

    r_btc = df["BTC-USD_close"].astype(float).pct_change().fillna(0.0)
    r_eth = df["ETH-USD_close"].astype(float).pct_change().fillna(0.0)

    position = position.reindex(df.index).fillna(0.0)
    return position.shift(1).fillna(0.0) * (r_btc - beta * r_eth)


def apply_turnover_cost(returns: pd.Series, position: pd.Series, fee_bps: float, slippage_bps: float) -> pd.Series:
    pos = position.reindex(returns.index).fillna(0.0)
    turnover = pos.diff().abs().fillna(0.0)
    cost_rate = (fee_bps + slippage_bps) / 1e4
    return returns - turnover * cost_rate

returns_raw = {
    "OU Pair": pair_portfolio_returns(df, PAIR_BETA_WINDOW, ou_pos),
    "Z-Score Pair": pair_portfolio_returns(df, PAIR_BETA_WINDOW, z_pos),
    "Trend Pair": pair_portfolio_returns(df, PAIR_BETA_WINDOW, trend_pos),
    "Mining Proxy Pair": pair_portfolio_returns(df, PAIR_BETA_WINDOW, mining_pos),
    "Ensemble Pair": pair_portfolio_returns(df, PAIR_BETA_WINDOW, ensemble_pos),
}

returns_net = {
    k: apply_turnover_cost(v, {"OU Pair": ou_pos, "Z-Score Pair": z_pos, "Trend Pair": trend_pos, "Mining Proxy Pair": mining_pos, "Ensemble Pair": ensemble_pos}[k], FEE_BPS, SLIPPAGE_BPS)
    for k, v in returns_raw.items()
}

summary = build_summary_table({
    k: {
        "returns": returns_net[k],
        "position": {"OU Pair": ou_pos, "Z-Score Pair": z_pos, "Trend Pair": trend_pos, "Mining Proxy Pair": mining_pos, "Ensemble Pair": ensemble_pos}[k],
    }
    for k in returns_net
})

summary


In [ ]:
# Benchmarks: BTC buy & hold and ETHA buy & hold
etha = pd.read_csv('../../Market Data/Crypto Data/ETHA.csv')
etha['Date'] = pd.to_datetime(etha['Start'])
etha = etha.sort_values('Date').set_index('Date')
etha_close = pd.to_numeric(etha['Close'], errors='coerce').replace([np.inf, -np.inf], np.nan).dropna()

btc_bh = df['BTC-USD_close'].astype(float).pct_change().fillna(0.0)
etha_bh = etha_close.pct_change().reindex(df.index).fillna(0.0)

eq = {
    'Ensemble Pair': (1.0 + returns_net['Ensemble Pair'].fillna(0.0)).cumprod(),
    'BTC Buy&Hold': (1.0 + btc_bh).cumprod(),
    'ETHA Buy&Hold': (1.0 + etha_bh).cumprod(),
}

plt.figure(figsize=(12,5))
for name, curve in eq.items():
    plt.plot(curve.index, curve.values, label=name)
plt.yscale('log')
plt.title('Pair Ensemble vs BTC / ETHA Buy&Hold')
plt.legend()
plt.show()
